In [5]:
using CondensedMatterSOS
@spin σ[1:2]

┌ Info: Precompiling CondensedMatterSOS [676dea7e-7535-42cf-a5c4-14732a6b7aba]
└ @ Base loading.jl:1260


((CondensedMatterSOS.SpinVariable[σˣ₁, σˣ₂], CondensedMatterSOS.SpinVariable[σʸ₁, σʸ₂], CondensedMatterSOS.SpinVariable[σᶻ₁, σᶻ₂]),)

In [6]:
function hamiltonian(σ)
    σx, σy, σz = σ
    N = length(σx)
    return -sum(σx[n] * σx[n+1] for n in 1:(N-1)) - σx[N] * σx[1] - sum(σz)
end
H = 1.0 * hamiltonian(σ)

(-2.0 + 0.0im)σˣ₁σˣ₂ + -σᶻ₁ + -σᶻ₂

In [53]:
import MultivariatePolynomials
const MP = MultivariatePolynomials
v = [1, σx[1], σx[2], σy[1], σy[2], σz[1], σz[2]]

7-element Array{CondensedMatterSOS.SpinTerm{Int64},1}:
 1
 σˣ₁
 σˣ₂
 σʸ₁
 σʸ₂
 σᶻ₁
 σᶻ₂

min_σ H(σ) = -2√2

max γ s.t. H >= γ -> H - γ = v' Q v

In [8]:
monos = CondensedMatterSOS.MP.monovec(v)

7-element Array{CondensedMatterSOS.SpinMonomial,1}:
 σˣ₁
 σʸ₁
 σᶻ₁
 σˣ₂
 σʸ₂
 σᶻ₂
 1

In [9]:
function _mat(x::Vector, n)
    Q = Matrix{eltype(x)}(undef, n, n)
    k = 0
    for j in 1:n
        for i in 1:j
            k += 1
            Q[i, j] = Q[j, i] = x[k]
        end
    end
    return Q
end
import MathOptInterface
const MOI = MathOptInterface
function _mapcoef(op, f::MOI.ScalarAffineFunction)
    MOI.ScalarAffineFunction([
        MOI.ScalarAffineTerm(op(t.coefficient), t.variable_index) for t in f.terms
    ], op(f.constant))
end

_mapcoef (generic function with 1 method)

# Real

In [26]:
using MosekTools
optimizer = Mosek.Optimizer()
n = length(v)
x, cx = MOI.add_constrained_variables(optimizer, MOI.PositiveSemidefiniteConeTriangle(n))
γ = MOI.SingleVariable(MOI.add_variable(optimizer))
F = MOI.ScalarAffineFunction{Complex{Float64}}
Q = _mat(F.(MOI.SingleVariable.(x)), n)
gram = sum(Q[i, j] * (1.0 * monos[i] * monos[j]) for i in 1:n for j in 1:n)
z = H - convert(F, γ) - gram
for c in MP.coefficients(z)
    for a in [_mapcoef(real, c), _mapcoef(imag, c)]
        MOI.Utilities.normalize_and_add_constraint(optimizer, a, MOI.EqualTo(0.0))
    end
end
MOI.set(optimizer, MOI.ObjectiveSense(), MOI.MAX_SENSE)
MOI.set(optimizer, MOI.ObjectiveFunction{MOI.SingleVariable}(), γ)
MOI.optimize!(optimizer)

MOSEK error 1001: The license has expired.

*** A FLEXlm error occurred. FLEXlm reported:

Feature has expired.
Feature:       PTS
Expire date:   14-jun-2020
License path:  /home/blegat/mosek/mosek.lic:
FlexNet Licensing error:-10,32

*** end of FLEXlm report.



Mosek.MosekError: Mosek.MosekError(1001, "The license has expired.")

# Complex

In [36]:
using CSDP
optimizer = MOI.instantiate(CSDP.Optimizer, with_bridge_type=Float64)
n = length(v)
x, cx = MOI.add_constrained_variables(optimizer, MOI.PositiveSemidefiniteConeTriangle(2n))
X = _mat(MOI.SingleVariable.(x), 2n)
X11 = X[1:n, 1:n]
X22 = X[n .+ (1:n), n .+ (1:n)]
X12 = X[1:n, n .+ (1:n)]
for j in 1:n
    for i in 1:j
        MOI.add_constraint(optimizer, MOI.Utilities.operate(-, Float64, X11[i, j], X22[i, j]), MOI.EqualTo(0.0))
        if i == j
            MOI.add_constraint(optimizer, convert(MOI.ScalarAffineFunction{Float64}, X12[i, j]), MOI.EqualTo(0.0))
        else
            MOI.add_constraint(optimizer, MOI.Utilities.operate(+, Float64, X12[i, j], X12[j, i]), MOI.EqualTo(0.0))
        end
    end
end
γ = MOI.SingleVariable(MOI.add_variable(optimizer))
F = MOI.ScalarAffineFunction{Complex{Float64}}
Q_real = F.(X11)
Q_comp = F.(X12)
Q = Q_real + (-1.0*im) * Q_comp
gram = sum(Q[i, j] * (1.0 * monos[i] * monos[j]) for i in 1:n for j in 1:n)
z = H - convert(F, γ) - gram
duals = Dict()
for term in MP.terms(z)
    duals[MP.monomial(term)] = map([real, imag]) do f
        a = _mapcoef(f, MP.coefficient(term))
        MOI.Utilities.normalize_and_add_constraint(optimizer, a, MOI.EqualTo(0.0))
    end
end
MOI.set(optimizer, MOI.ObjectiveSense(), MOI.MAX_SENSE)
MOI.set(optimizer, MOI.ObjectiveFunction{MOI.SingleVariable}(), γ)
MOI.optimize!(optimizer)

CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 1.00e+00 Pobj: -5.6507691e+01 Ad: 4.01e-01 Dobj: -1.7634637e+00 
Iter:  2 Ap: 1.00e+00 Pobj: -5.5377524e+01 Ad: 9.54e-01 Dobj: -3.1515529e-01 
Iter:  3 Ap: 1.00e+00 Pobj: -2.4963595e+01 Ad: 8.83e-01 Dobj: -5.2930472e-01 
Iter:  4 Ap: 1.00e+00 Pobj: -6.2669103e+00 Ad: 7.62e-01 Dobj: -1.2742768e+00 
Iter:  5 Ap: 1.00e+00 Pobj: -3.5102614e+00 Ad: 8.76e-01 Dobj: -2.3614414e+00 
Iter:  6 Ap: 1.00e+00 Pobj: -3.0661231e+00 Ad: 8.87e-01 Dobj: -2.7653758e+00 
Iter:  7 Ap: 1.00e+00 Pobj: -2.8484566e+00 Ad: 8.92e-01 Dobj: -2.8141101e+00 
Iter:  8 Ap: 1.00e+00 Pobj: -2.8301498e+00 Ad: 9.38e-01 Dobj: -2.8268972e+00 
Iter:  9 Ap: 1.00e+00 Pobj: -2.8285576e+00 Ad: 9.86e-01 Dobj: -2.8283564e+00 
Iter: 10 Ap: 9.75e-01 Pobj: -2.8284373e+00 Ad: 9.98e-01 Dobj: -2.8284264e+00 
Iter: 11 Ap: 1.00e+00 Pobj: -2.8284276e+00 Ad: 1.00e+00 Dobj: -2.8284271e+00 
Iter: 12 Ap: 9.58e-01 Pobj: -2.8284271e+00 Ad: 9.60e-

15.998771905899048

In [30]:
@show MOI.get(optimizer, MOI.TerminationStatus())

MOI.get(optimizer, MOI.TerminationStatus()) = MathOptInterface.OPTIMAL


OPTIMAL::TerminationStatusCode = 1

In [31]:
-2 * √2

-2.8284271247461903

In [32]:
MOI.get(optimizer, MOI.ObjectiveValue())

-2.828427145315304

In [33]:
MOI.get(optimizer, MOI.SolveTime())

15.215893030166626

In [34]:
Mfull = round.(_mat(MOI.get(optimizer, MOI.ConstraintDual(), cx), 2n), digits=6)
M11 = Mfull[1:n, 1:n]
M12 = Mfull[1:n, n .+ (1:n)]
M12 + im * M12

7×7 Array{Complex{Float64},2}:
       0.0+0.0im       0.353553+0.353553im  …  0.0+0.0im  0.0+0.0im
 -0.353553-0.353553im       0.0+0.0im          0.0+0.0im  0.0+0.0im
       0.0+0.0im            0.0+0.0im          0.0+0.0im  0.0+0.0im
       0.0+0.0im           -0.0-0.0im          0.0+0.0im  0.0+0.0im
      -0.0-0.0im            0.0+0.0im          0.0+0.0im  0.0+0.0im
       0.0+0.0im            0.0+0.0im       …  0.0+0.0im  0.0+0.0im
       0.0+0.0im            0.0+0.0im          0.0+0.0im  0.0+0.0im

In [47]:
function _moment_matrix(μ::MultivariateMoments.Measure{T}, X) where T
    function getmom(i, j)
        x = X[i] * X[j]
        for m in moments(μ)
            if monomial(m) == monomial(x)
                return MP.coefficient(x) * moment_value(m)
            end
        end
        throw(ArgumentError("μ does not have the moment $(x)"))
    end
    return MultivariateMoments.MomentMatrix{T}(getmom, X)
end

_moment_matrix (generic function with 1 method)

In [54]:
using MultivariateMoments
a = Complex{Float64}[]
m = MP.monomialtype(H)[]
for (key, value) in duals
    push!(m, key)
    real_dual = MOI.get(optimizer, MOI.ConstraintDual(), value[1])
    comp_dual = MOI.get(optimizer, MOI.ConstraintDual(), value[2])
    push!(a, real_dual + im * comp_dual)
end
μ = measure(a, m)
ν = _moment_matrix(μ, v)

MomentMatrix{Complex{Float64},MultivariateBases.MonomialBasis{CondensedMatterSOS.SpinMonomial,Array{CondensedMatterSOS.SpinMonomial,1}}}(Complex{Float64}[1.0000000007926881 + 0.0im 0.0 - 0.7071061572962712im … 0.0 - 0.0im 0.0 - 0.0im; 0.0 - 0.7071061572962712im 1.0000000007926881 + 0.0im … 0.0 - 0.0im 0.0 - 0.0im; … ; 0.0 - 0.0im 0.0 - 0.0im … 1.0000000007926881 + 0.0im 0.7071061572962712 + 0.0im; 0.0 - 0.0im 0.0 - 0.0im … 0.7071061572962712 + 0.0im 1.0000000007926881 + 0.0im], MultivariateBases.MonomialBasis{CondensedMatterSOS.SpinMonomial,Array{CondensedMatterSOS.SpinMonomial,1}}(CondensedMatterSOS.SpinMonomial[σˣ₁, σʸ₁, σᶻ₁, σˣ₂, σʸ₂, σᶻ₂, 1]), nothing)

In [56]:
round.(ν.Q, digits = 1)

7×7 Array{Complex{Float64},2}:
 1.0+0.0im   0.0-0.7im   0.0-0.0im  …   0.0-0.0im   0.0-0.0im  0.0-0.0im
 0.0-0.7im   1.0+0.0im  -0.0+0.0im     -0.7-0.0im   0.0-0.0im  0.0-0.0im
 0.0-0.0im  -0.0+0.0im   1.0+0.0im      0.0-0.0im   0.5+0.0im  0.7+0.0im
 0.7+0.0im   0.0+0.0im   0.0-0.0im      0.0-0.7im   0.0-0.0im  0.0-0.0im
 0.0-0.0im  -0.7-0.0im   0.0-0.0im      1.0+0.0im  -0.0+0.0im  0.0-0.0im
 0.0-0.0im   0.0-0.0im   0.5+0.0im  …  -0.0+0.0im   1.0+0.0im  0.7+0.0im
 0.0-0.0im   0.0-0.0im   0.7+0.0im      0.0-0.0im   0.7+0.0im  1.0+0.0im

In [10]:
using LinearAlgebra
Γ = Hermitian([
    1 0 0   0     0     1/2 1/2
    0 1 1/2 1/2im 0     0   0
    0 0 1   0     1/2im 0   0
    0 0 0   1    -1/2   0   0
    0 0 0   0     1     0   0
    0 0 0   0     0     1   1
    0 0 0   0     0     0   1
])

7×7 Hermitian{Complex{Float64},Array{Complex{Float64},2}}:
 1.0+0.0im  0.0+0.0im  0.0+0.0im  …   0.0+0.0im  0.5+0.0im  0.5+0.0im
 0.0-0.0im  1.0+0.0im  0.5+0.0im      0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0-0.0im  0.5-0.0im  1.0+0.0im      0.0-0.5im  0.0+0.0im  0.0+0.0im
 0.0-0.0im  0.0+0.5im  0.0-0.0im     -0.5+0.0im  0.0+0.0im  0.0+0.0im
 0.0-0.0im  0.0-0.0im  0.0+0.5im      1.0+0.0im  0.0+0.0im  0.0+0.0im
 0.5-0.0im  0.0-0.0im  0.0-0.0im  …   0.0-0.0im  1.0+0.0im  1.0+0.0im
 0.5-0.0im  0.0-0.0im  0.0-0.0im      0.0-0.0im  1.0-0.0im  1.0+0.0im